In [1]:
# Own Packages
from Masterarbeit_utils.model_utils_seq_class import load_and_modify_model, get_tokenizer


# Site-Packages
import dask.dataframe as dd
import torch
import psutil
import os
import sys
import pickle as pk
import pandas as pd
import numpy as np

from transformers import AutoTokenizer, OPTForSequenceClassification
from tokenizers.processors import TemplateProcessing
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding
from torch.utils.data import Dataset
sys.version, sys.executable

2023-08-23 14:38:11.465704: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-23 14:38:11.484295: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-23 14:38:11.890920: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


('3.10.0 (default, Jul 12 2023, 08:49:30) [GCC 12.2.0]',
 '/home/worker/.pyenv/versions/3.10.0/bin/python3.10')

In [2]:
choices = ['calculate all', 'ask for userinput', 'just calculate needed']
calculation_profile =  choices[2]
calculation_profile

'just calculate needed'

In [6]:
"""
The Paths to important folders have to be changed for your system.
"""

# Name of this experiment
model_name = 'gal_125_seq_3'

# This folder will be created and filled with txt.files for each sample after you run the Pytorch Dataset Notebook
dataset_folder = f'data/dataset_samples'

# The folder at which the model will be saved. This folder has to be created for your system 
model_folder = f'data/models/{model_name}'
os.makedirs(model_folder, exist_ok=True)


# Folder in which the tokenizer will be saved
tokenizer_folder = f'data/tokenizers/{model_name}'
os.makedirs(tokenizer_folder, exist_ok=True)

# Folder at which all pickle files are stored. This folder is fixed for this project and should not be changed
dump_dir = r'PK_DUMP'

# Model parameters 
'''
mini	125 M
base	1.3 B
standard	6.7 B
large	30 B
huge	120 B'''
base_model_name = 'mini'

# All new Torch-objects will be by default in this dtype
# if default_type = float16 fp16 must be False
default_dtype = torch.bfloat16
torch.backends.cuda.matmul.allow_tf32 = True
torch.set_default_dtype(default_dtype)

# Default device on which the model will be loaded
default_device = 'cuda:0'

# Number of GPUs the model will be parallelised to 
num_gpus = 1
# If you change 'default_device' to 'cpu', make sure to set num_gpus to zero.
if default_device == 'cpu':
    num_gpus = 0

tensor_parallel = False


# Creating the Tokenizer

In [7]:
if calculation_profile == choices[0]:
    i = 'y'
elif calculation_profile == choices[1]:  
    i = input("This creates a new tokenizer instance and saves it, if you want to proceed write y: ")
else:
    i = 'n'

if i != 'y' and os.path.isfile(f'{tokenizer_folder}/tokenizer.json'):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_folder)
    n_f_terms = len(tokenizer) - tokenizer.vocab_size
    print('Loaded Tokenizer from serialized instance!')    
    print(f'There are {n_f_terms} different F-Terms in the whole Dataset!')
    # the padding_side attribute is not save with the custom tokenizer and has to be reset after loading it
    tokenizer.padding_side = 'left'
    
else:
    print('Generate new Tokenizer')
    # Loads a pretrained Tokenizer for the galactica model and adds an additional token for each F-Term
    tokenizer = get_tokenizer(dump_dir)
    
    # The Tokenizer contained initially 50000 Tokens which are stored as the vocab-size.
    # The vocab_size attribute is not updated when the additional tokens are added to the tokenizer
    n_f_terms = len(tokenizer) - tokenizer.vocab_size
    tokenizer.save_pretrained(tokenizer_folder)
    print(f'There are {n_f_terms} different F-Terms in the whole Dataset!')

tokenizer.padding_side

Generate new Tokenizer
There are 378166 different F-Terms in the whole Dataset!


'left'

# Loading The Model

In [8]:

device_map=None

max_memory = {}
if num_gpus > 0:

    # based on https://github.com/huggingface/accelerate/blob/5315290b55ea9babd95a281a27c51d87b89d7c85/src/accelerate/utils/modeling.py#L274

    for i in range(num_gpus):
         _ = torch.tensor([0], device=i)

    for i in range(num_gpus):
        max_memory[i] = torch.cuda.mem_get_info(i)[0]
    device_map = "auto"
max_memory["cpu"] = psutil.virtual_memory().available

model = OPTForSequenceClassification.from_pretrained(f'{model_folder}/checkpoint-74039', torch_dtype=default_dtype, low_cpu_mem_usage=True,
                                           device_map=device_map, max_memory=max_memory)



OSError: Can't load the configuration of 'data/models/gal_125_seq_3/checkpoint-74039'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'data/models/gal_125_seq_3/checkpoint-74039' is the correct path to a directory containing a config.json file

# Loading the Datasets and the Tokenizer

In [6]:
class JapPatDataset(Dataset):
    """Dataset containing Japanese patents and their F-Term classification.
    This variant is adapted for sequence classification and returns the f_terms as a list of labels"""
    def __init__(self, data_folder, tokenizer):
        """
        data_folder: path to folder containing the text samples
        tokenizer: tokenizer instance with added additional Tokens for F-Terms
        """
        super(Dataset).__init__()
        self.data_folder = data_folder
        # This has to be manually set to the ammount of files in the 'dataset_samples' folder. Calculating the number of files in this folder would take forever.
        # A to low number would lead to samples missing from the dataset.
        # A to high number would raise a FileNotFound error.
        self.l = len(os.listdir(data_folder))
        self.start_f_term_token = '<START F-TERMS>'
        self.tokenizer = tokenizer
        
    def __len__(self):
        return self.l
    
    def __getitem__(self, idx):
        try:
            with open(f'{self.data_folder}/{idx}.txt', 'r', encoding='utf-8') as f:
                item = f.read()
        except FileNotFoundError:
            raise FileNotFoundError

        #tokenizing the whole sample which will be later split into tokens and labels
        tokenized = self.tokenizer(item)
        tokenized.pop('token_type_ids')
        attention_mask = tokenized.pop('attention_mask')
        tokens = tokenized.pop('input_ids')
        
        tokens = torch.tensor(tokens)
        # separating the abstract text tokens from the f_terms
        input_ids = tokens[tokens < 50000].tolist()
        f_term_ids = tokens[tokens >= 50002] - 50000
        # rescaling the attention_mask to the shorter sequence
        attention_mask = attention_mask[:len(input_ids)]

        # creating a multi hot vector as the label 
        n_f_terms = len(self.tokenizer) - self.tokenizer.vocab_size
        labels = torch.zeros([n_f_terms])
        labels[f_term_ids] = 1
        return {'input_ids': input_ids, 'attention_mask': attention_mask,   'labels':labels.tolist()}

In [7]:

train_dataset = JapPatDataset(f'{dataset_folder}/train', tokenizer)
validation_dataset = JapPatDataset(f'{dataset_folder}/validation', tokenizer)

# Loading a dict that contains the definitions of the f-terms
with open(f'{dump_dir}/full_descriptions.pk', 'rb') as f:
    full_descriptions_dict = pk.load(f)

In [8]:
def classic_accurracy(batch, model):
    """
    Classic prediction accuracy metric. 
    This function should be applied to a batch of samples,
    which were tokenized by a tokenizer instance.
    This function returns the procentual accuracy metric as well as the total number of correct predictions
    and the total number of predictions in this batch

    :batch: batch of samples from validation dataset
    :model: model which should be testet
    :top_k: top k predictions which should be investigated for a correct result.
    """
    #input_ids = batch['input_ids']
    #attention_mask = batch['attention_maks']
    labels = batch.pop('labels')
    
    with torch.no_grad():
        model.eval()
        logits = model(**batch, output_hidden_states=False, return_dict=True)['logits']

    logits, idx_logits = torch.sort(logits, descending=True, dim=-1)
    labels, idx_labels = torch.sort(labels, descending=True, dim=-1)
    n_preds = labels.sum(-1)
    
    corr = 0
    pred = 0
    for targets, predictions, n_pred in zip(idx_labels, idx_logits, n_preds):
        targets = targets[:int(n_pred.item())]
        predictions = predictions[:int(n_pred.item())]

        tar_pred = torch.cat([targets, predictions], -1)
        occ = torch.nn.functional.one_hot(tar_pred).sum(dim=-2)
        
        corr += len(occ[occ ==2])
        pred += n_pred

    return 100*corr/pred , corr, pred
        
        
class Batch_DataLoader():
    """
    This class converts a dataset to a iterable dataloader, which loads padded patches of data.   
    """
    def __init__(self,
                 dataset, 
                 batchsize=10,
                 datacollator=DataCollatorWithPadding(tokenizer, return_tensors='pt')):

        self.dataset = dataset
        self.batchsize = batchsize
        self.l = len(dataset)//batchsize + 1
        self.datacollator = datacollator
        self.current = 0

    def __len__(self):
        return self.l

    def __iter__(self):
        self.current = 0
        return self

    def __next__(self):
        batch = [self.dataset[i] for i in range(self.current, self.current+self.batchsize)]
        batch = self.datacollator(batch)
        self.current += self.batchsize
        return batch
        

In [9]:
def accuracy_on_dataset(dataset, model, batch_size=10):
    loader = Batch_DataLoader(dataset, batch_size)
    n_pred = 0
    n_corr = 0
    for i, batch in enumerate(loader):
        acc, corr, pred = classic_accurracy(batch, model)
        n_pred += pred
        n_corr += corr
        print(f'batch_acc: {acc:.2f}%, total_acc: {100*n_corr/n_pred:.2f}% batch {i}/{len(loader)}', end ='\r')
    return n_pred, n_corr
    

n_pred, n_corr = accuracy_on_dataset(validation_dataset, model, batch_size=40)

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


KeyboardInterrupt: 